1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.
2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?
3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?
4. What is the correlation between the overall score variable and the 1st principal component? 
5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings
from scipy.stats.stats import ttest_ind
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from scipy.stats import boxcox

# warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [2]:
cols = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

states = df["STATE"].unique()

for state in states:
    df.loc[df["STATE"] == state, cols] = df.loc[df["STATE"] == state, cols].interpolate()

# we drop the null values after interpolation
df.dropna(inplace=True)

In [3]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,57098.0,58305.0,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,62793.0,60514.0,42041.0,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,33855.0,37064.0,27613.0,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,435217.0,406456.0,298669.0,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,52524.0,52269.0,37179.0,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154


In [4]:
df2 = df.copy()

## 1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.

In [6]:
df2["AVG_SCORE"] = (df2["GRADES_4_G"]*((df2["AVG_MATH_4_SCORE"] + df2["AVG_READING_4_SCORE"])*0.5) + df2["GRADES_8_G"]
                                 * ((df2["AVG_MATH_8_SCORE"] + df2["AVG_READING_8_SCORE"])*0.5))/(df2["GRADES_4_G"] + df2["GRADES_8_G"])

## 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [7]:
df2.corr()

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,AVG_SCORE
YEAR,1.000000,-0.001166,0.145326,0.146553,0.127570,0.151586,0.136971,0.144266,0.155056,0.119910,...,-0.005636,0.054619,-0.003866,0.016611,0.006251,0.547233,0.373242,0.254820,0.194247,0.404744
ENROLL,-0.001166,1.000000,0.925240,0.935013,0.923151,0.852341,0.927807,0.885390,0.931539,0.967845,...,0.999020,0.986636,0.999014,0.997492,0.999412,-0.020359,-0.041960,-0.069275,-0.004275,-0.042762
TOTAL_REVENUE,0.145326,0.925240,1.000000,0.927106,0.972620,0.964204,0.999068,0.989470,0.994620,0.944086,...,0.922690,0.944928,0.916786,0.936055,0.923363,0.102264,0.070952,0.047518,0.016828,0.076277
FEDERAL_REVENUE,0.146553,0.935013,0.927106,1.000000,0.922190,0.843644,0.927395,0.887882,0.931601,0.952760,...,0.932554,0.954865,0.928955,0.944021,0.934809,0.039994,-0.015730,-0.076318,0.005283,-0.014175
STATE_REVENUE,0.127570,0.923151,0.972620,0.922190,1.000000,0.878809,0.969617,0.952093,0.969140,0.946388,...,0.920535,0.951502,0.913856,0.936835,0.919243,0.059873,0.027388,-0.008741,0.012840,0.030668
LOCAL_REVENUE,0.151586,0.852341,0.964204,0.843644,0.878809,1.000000,0.965184,0.970662,0.955246,0.865485,...,0.850038,0.861670,0.844901,0.859777,0.852317,0.150224,0.127703,0.126788,0.021968,0.135470
TOTAL_EXPENDITURE,0.136971,0.927807,0.999068,0.927395,0.969617,0.965184,1.000000,0.990255,0.992988,0.943702,...,0.925176,0.944716,0.919309,0.937669,0.925799,0.099817,0.068509,0.044556,0.017619,0.074155
INSTRUCTION_EXPENDITURE,0.144266,0.885390,0.989470,0.887882,0.952093,0.970662,0.990255,1.000000,0.974399,0.902173,...,0.882167,0.901894,0.875932,0.895384,0.882302,0.105580,0.076477,0.068965,0.023135,0.087386
SUPPORT_SERVICES_EXPENDITURE,0.155056,0.931539,0.994620,0.931601,0.969140,0.955246,0.992988,0.974399,1.000000,0.952282,...,0.929005,0.953908,0.922631,0.942777,0.929590,0.106347,0.073325,0.046760,0.019423,0.079002
OTHER_EXPENDITURE,0.119910,0.967845,0.944086,0.952760,0.946388,0.865485,0.943702,0.902173,0.952282,1.000000,...,0.965931,0.979411,0.963881,0.973864,0.967555,0.050671,0.002578,-0.028763,0.012918,0.010959


#### Instruction expenditure is the most correlated.

## 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df2[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=4)
df2["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175  0.03343929 0.02005189 0.00349132]


#### 0.9430175

In [19]:
df2.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,AVG_SCORE,pca_1
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441,235.106342,-0.939455
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,42041.0,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638,238.321404,-0.802867
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,27613.0,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590,240.773675,-1.219663
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,298669.0,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451,234.719834,3.646088
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,37179.0,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154,247.750273,-0.958451


## 4. What is the correlation between the overall score variable and the 1st principal component? 

In [21]:
df2[["AVG_SCORE", "pca_1", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

#df2[["AVG_SCORE", "pca_1"]].corr()

,AVG_SCORE,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
AVG_SCORE,1.000000,0.044596,0.074155,0.087386,0.079002,0.010959,-0.005389
pca_1,0.044596,1.000000,0.992054,0.968606,0.988345,0.971343,0.955792
TOTAL_EXPENDITURE,0.074155,0.992054,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.968606,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.988345,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.971343,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.955792,0.925784,0.884798,0.911245,0.918188,1.000000


#### 0.044596

## 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

No. There is already high correlation between